<a href="https://colab.research.google.com/github/Prajwal-5012/Smart_Robot_Navigation_System/blob/main/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Smarter Robot Navigation**

In [2]:
import sys
import heapq

print("Provide the size of maze(first row value and then column)")
n, m = map(int, input().split())

g = [] #For Maze
vis = [] #stores whether a tile is visited or not.
dist = [] #stores the distance of tile from starting tile
par = [] #Store the coordinate of the parent node
path = [] #stores the final path
moves = [[1, 0], [0, 1], [-1, 0], [0, -1]] #Moves of robot. 1 and 0  basically indicate the change in coordinate from the current position when the robot moves right, down, left and up.

#function to check if the coordinate lies inside the maze or not.
def check(x, y):
    return 0 <= x < n and 0 <= y < m

#Node definition
class Node:
    def __init__(self, wt, direction, c, p):
        self.wt = wt #stores weight of edge
        self.direction = direction #indicates the orientation of robot. 1 indicates it is oriented at 45 degree in a cell and 0 otherwise.
        self.c = c #stores the current coordinate of the cell
        self.p = p #stores the coordinate of the parent cell

    def __lt__(self, other):
        return self.wt < other.wt

#function that provide the weight of the edge between the two cells based on the orientation and previous path.
def weight(x, y, nn):
    if nn.direction == 0:
        if abs(x - nn.p[0]) == 1 and abs(y - nn.p[1]) == 1:
            return 1
        if abs(x - nn.c[0]) == 1 and abs(y - nn.c[1]) == 1:
            return 0.5
        return 0
    elif nn.direction == 1:
        if abs(x - nn.p[0]) == 2 and abs(y - nn.p[1]) == 2:
            return 0
        if abs(x - nn.c[0]) == 1 and abs(y - nn.c[1]) == 1:
            return 1
        return 0.5
    return 0

def solve():
    global n, m, g, vis, dist, par, path, moves

    #Taking the input for Maze
    print("Enter your Maze")
    g = [list(map(int, input().split())) for _ in range(n)]
    vis = [[0] * m for _ in range(n)] #Initializing all cells with 0 indicating they are not visited
    dist = [[1e9] * m for _ in range(n)] #Initializing the distance of all cells from starting point with 1e9(infinity) indicating not reachable
    par = [[(i, j) for j in range(m)] for i in range(n)] #Initializing the parent coordinate of each cell with itself indicating they do not have any initial parent node

    #Taking the starting and end points of Robot
    print("Enter the starting coordinates")
    start = list(map(int, input().split()))
    print("Enter the ending coordinates")
    end = list(map(int, input().split()))

    #Legal Directions
    print("Enter 0 if robot can moves in only 4 direction else enter 1 if all directions are allowed")
    all_direction = int(input())
    #If all direction are allowed update the moves array with further directions, which is right-down, left-down, left-up and right-up.
    if all_direction:
        moves.extend([[1, 1], [-1, 1], [-1, -1], [1, -1]])

    is_possible = False #Indicates that the ending point is not reachable from the starting point
    dist[start[0]][start[1]] = 0 #setting the distance of starting cell to zero

    #Initializing the list and Inserting the starting node into it
    #Here we are inserting the two different configurations of the starting node. direction with zero indicates its orientation is straight,
    #whereas 1 indicates it is oriented at 45 degree.
    pq = []
    heapq.heappush(pq, Node(0, 0, start, start))
    heapq.heappush(pq, Node(0, 1, start, start))

    #Dijkstra Algorithm
    while pq:
        top = heapq.heappop(pq)
        vis[top.c[0]][top.c[1]] = 1  #Marking the node as visited as we have arrived at the shortest distance of the node.
        par[top.c[0]][top.c[1]] = top.p #Updating it with parent coordinate.

        #checking if the given cell is the ending point or not and terminating the Dijkstra algorithm if so.
        if top.c[0] == end[0] and top.c[1] == end[1]:
            is_possible = True
            break

        #exploring all poosible cells from current cell
        for move in moves:
            x, y = top.c[0] + move[0], top.c[1] + move[1]
            #checking if the next possible cell lies within the grid and whether it is a white tile or black tile
            if check(x, y) and vis[x][y] == 0 and g[x][y] != 0:
                dd = 0; #indicates the orientation of the next cell.
                wtt = weight(x, y, top)

                #Changing the orientation according to the move
                if top.direction == 0 and wtt == 0.5:
                  dd = 1
                elif top.direction == 1:
                    if wtt == 0 or wtt == 1:
                        dd = 1
                wtt += 1 #increasing the weight by 1 indicating forward movement
                #checking if the current distance is less than the previous distance and if so updating it with the smaller distance and adding the node to heap
                if dist[x][y] > dist[top.c[0]][top.c[1]] + wtt:
                    dist[x][y] = dist[top.c[0]][top.c[1]] + wtt
                    heapq.heappush(pq, Node(dist[x][y], dd, [x, y], top.c))

    print()
    print("Output:")
    if is_possible:
        print("True")

        #Code to print distance matrix and parent matrix
        # for i in range(n):
        #     for j in range(m):
        #         if dist[i][j] == 1e9:
        #             print("I", end=" ")
        #         else:
        #             print(dist[i][j], end=" ")
        #     print()

        # print()
        # print()

        # for i in range(n):
        #     for j in range(m):
        #         print(f"({par[i][j][0]},{par[i][j][1]}) ", end="")
        #     print()

        # print()
        # print()

        #code to print path.
        #Backtracking the path with the help of parent coordinates stored in the parent matrix until coordinate stored in the parent matix equals actual coordinate indicating
        # it has no further parent(i.e. source)
        node = tuple(end)
        while par[node[0]][node[1]] != node:
            path.append(node)
            node = par[node[0]][node[1]]
        path.append(tuple(start))
        #reversing the tuple to get the actual path and printing it
        path.reverse()
        path.pop()
        print("Following is the possible path the robot can take to reach the end point:")
        print("Path: [", end="")
        for v in path:
            print(f"({v[0]},{v[1]}) ,", end=" ")
        print(f"({end[0]},{end[1]})]")
    else:
        print("False")

if __name__ == "__main__":
    solve()


Provide the size of maze(first row value and then column)
5 5
Enter your Maze
1 1 1 1 1
1 1 0 0 1
1 1 1 0 1
1 1 0 0 1
1 1 1 1 1
Enter the starting coordinates
2 2
Enter the ending coordinates
4 4
Enter 0 if robot can moves in only 4 direction else enter 1 if all directions are allowed
0

Output:
True
Following is the possible path the robot can take to reach the end point:
Path: [(2,2) , (2,1) , (3,1) , (4,1) , (4,2) , (4,3) , (4,4)]
